In [1]:
import re
import itertools
from __future__ import print_function
import pandas as pd
import datetime as dt
import numpy as np
import nltk
import string
import scipy.sparse as sparse
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('seaborn-white')

re_spaces = re.compile(r'\s+')

from mpl_toolkits.mplot3d import axes3d

C:\Users\Shivi\Anaconda3\envs\MachineLearning\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
def cleaned_reviews(x):
    #return(''.join(re.sub('[^a-zA-Z_]', ' ', x)))
    return (x.replace(':',' ').lower().replace('\r',' ').replace('!',' ').replace('\n',' '))

In [3]:
import spacy
import en_core_web_sm

nlp = en_core_web_sm.load()

In [12]:
from nltk.corpus import stopwords
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

from nltk.tokenize import RegexpTokenizer
def get_bigram_likelihood(statements, freq_filter=3, nbest=200):
    """
    Returns n (likelihood ratio) bi-grams from a group of documents
    :param        statements: list of strings
    :param output_file: output path for saved file
    :param freq_filter: filter for # of appearances in bi-gram
    :param       nbest: likelihood ratio for bi-grams
    """

    #words = list()
    #tokenize sentence into words
    #for statement in statements:
        # remove non-words
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(statements)

    bigram_measures = nltk.collocations.BigramAssocMeasures()
    bigram_finder = BigramCollocationFinder.from_words(words)

    # only bi-grams that appear n+ times
    bigram_finder.apply_freq_filter(freq_filter)

    # TODO: use custom stop words
    bigram_finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in nltk.corpus.stopwords.words('english'))

    #bigram_results = bigram_finder.nbest(bigram_measures.likelihood_ratio, nbest)

    return bigram_finder.nbest(bigram_measures.likelihood_ratio, nbest)

#bigram_finder.score_ngrams(bigram_measures.likelihood_ratio)

In [5]:
Bose = pd.read_csv(r'C:\Users\Shivi\PycharmProjects\BIA660D_Group_2_Project\Brandwise_files\BoseAudio_headphones.csv') 

In [6]:
Bose = Bose.rename(index=str, columns={"Unnamed: 0": "ID"})

In [7]:
Bose['Reviews'] = Bose['Reviews'].apply(lambda x : cleaned_reviews(str(x)))

In [8]:
bose_data = Bose['Reviews'].tolist()

In [9]:
bose_data = ''.join(bose_data)

In [ ]:
bose_data

In [13]:
bose_bigrams = get_bigram_likelihood(bose_data, freq_filter=3, nbest=200 )

In [ ]:
bose_bigrams

In [15]:
def sample_reviews(id):
    bigrams = [('better', 'portability'),('noise', 'cancellation'), ('battery', 'life'), ('sound', 'quality'), ('noise', 'canceling')]
    review_texts = Bose[Bose['ID'] == id]['Reviews'].values
    #review_texts = Bose['Reviews'].values
    sample_reviews = []
    for bigram in bigrams:
        sample_review_list = list(filter(lambda txt: " ".join(bigram) in txt, review_texts))
        num_reviews = len(sample_review_list)
        if num_reviews != 0:
            sample_review = sample_review_list[0]
            sample_review = sample_review.replace(" ".join(bigram), "****" + "_".join(bigram) + "****")
            start_index = sample_review.index("****")
            sample_text = sample_review[start_index - len(sample_review): start_index + len(sample_review)]
            sample_reviews.append(sample_text)
    return sample_reviews

In [16]:
Bose['Featured_reviews'] = Bose['ID'].apply(lambda x: sample_reviews(x))

In [ ]:
Bose['Featured_reviews']

In [18]:
def filter_reviews(x):
    features = ['noise_cancellation', 'battery_life', 'sound_quality', 'noise_canceling']
    for word in x:
        if any(feature in word for feature in features):
            return 'Yes'

In [19]:
Bose['Featured_Class'] = Bose['Featured_reviews'].apply(lambda x : filter_reviews(x))

In [20]:
Bose['Featured_Class'] = Bose['Featured_Class'].replace('None',np.nan)

In [21]:
df = Bose.dropna()

In [22]:
df = df.loc[:,['ID','Ratings','Reviews']]

In [23]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Shivi\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [24]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

C:\Users\Shivi\Anaconda3\envs\MachineLearning\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [25]:
def review_sentiment(string):
    sent = analyser.polarity_scores(string)
    return sent

In [26]:
df['Sentiments'] = df['Reviews'].apply(lambda x: review_sentiment(x))

In [27]:
df = pd.concat([df.drop(['Sentiments'], axis=1), df['Sentiments'].apply(pd.Series)], axis=1)

In [28]:
negativity = df['neg'].sum()
print(negativity)
positivity = df['pos'].sum()
print(positivity)
nuetrality = df['neu'].sum()
print(nuetrality)

29.678
84.773
407.551
